In [1]:
import pandas as pd
import numpy as np

# Importing the original dataset in SAS format
path = 'C:/Users/Kenneth/Desktop/18040394_KennethLauKhengSoong_Aug21/System Files/Original Data Files/ed2018_sas.sas7bdat'

# Selecting only required variables from the original dataset
df = pd.read_sas(path)
df1 = df[['WAITTIME','AGER','SEX','ARREMS','AMBTRANSFER','IMMEDR','PAINSCALE','HOSPCODE','PATCODE']]

# Writing out the dataset with only the chosen variables
df1.to_csv('C:/Users/Kenneth/Desktop/18040394_KennethLauKhengSoong_Aug21/System Files/Original Data Files/ED2018.csv', index=False)

In [2]:
# Reading the dataset with only chosen variables
ed18 = pd.read_csv('C:/Users/Kenneth/Desktop/18040394_KennethLauKhengSoong_Aug21/System Files/Original Data Files/ED2018.csv')

# Data cleaning
ed18['WAITTIME'] = ed18['WAITTIME'].replace([-9], np.NaN)
ed18['SEX'] = ed18['SEX'].replace([1], 0)
ed18['SEX'] = ed18['SEX'].replace([2], 1)
ed18['ARREMS'] = ed18['ARREMS'].replace([-8], np.NaN)
ed18['ARREMS'] = ed18['ARREMS'].replace([-9], np.NaN)
ed18['ARREMS'] = ed18['ARREMS'].replace([2], 0)
ed18['AMBTRANSFER'] = ed18['AMBTRANSFER'].replace([-8], np.NaN)
ed18['AMBTRANSFER'] = ed18['AMBTRANSFER'].replace([-9], np.NaN)
ed18['AMBTRANSFER'] = ed18['AMBTRANSFER'].replace([-7], 3)
ed18['IMMEDR'] = ed18['IMMEDR'].replace([0], 6)
ed18['IMMEDR'] = ed18['IMMEDR'].replace([-8], np.NaN)
ed18['IMMEDR'] = ed18['IMMEDR'].replace([-9], np.NaN)
ed18['PAINSCALE'] = ed18['PAINSCALE'].replace([-8], np.NaN)
ed18['PAINSCALE'] = ed18['PAINSCALE'].replace([-9], np.NaN)

In [3]:
# Printing the descriptive analysis
print("\nDescriptive Analysis of ED2018 Before Data Pre-processing\n")
descanalysis = ed18.describe(include = 'all')
print(descanalysis)

# Printing all variables with their number of missing values
print("\nNumber of Missing Values Found for Each Variable\n")
summiss = ed18.isnull().sum()
print(summiss)


Descriptive Analysis of ED2018 Before Data Pre-processing

           WAITTIME          AGER           SEX        ARREMS   AMBTRANSFER  \
count  17808.000000  20291.000000  20291.000000  19525.000000  19702.000000   
mean      35.647069      3.160909      0.452762      0.168195      2.845346   
std       69.276977      1.498474      0.497776      0.374049      0.388778   
min       -7.000000      1.000000      0.000000      0.000000      1.000000   
25%        6.000000      2.000000      0.000000      0.000000      3.000000   
50%       15.000000      3.000000      0.000000      0.000000      3.000000   
75%       38.000000      4.000000      1.000000      0.000000      3.000000   
max     1282.000000      6.000000      1.000000      1.000000      3.000000   

             IMMEDR     PAINSCALE      HOSPCODE       PATCODE  
count  15601.000000  12975.000000  20291.000000  20291.000000  
mean       3.549644      4.501118    116.323148     50.677591  
std        1.247968      3.717267   

In [4]:
from sklearn.impute import KNNImputer # Importing the K-Nearest Neighbor Package

imputer = KNNImputer() # Defining the Imputer

imputed_waittime = imputer.fit_transform(ed18) # Fitting the Imputer and Transforming it onto the Dataset
ed18_temp = pd.DataFrame(imputed_waittime) # Creating a temporary dataframe to store the imputed values
ed18_temp.columns = ed18.columns # Copying the columns from the main dataframe to the temporary dataframe

# Replacing the imputed values for variables with missing values from the temporary dataframe to the existing dataframe
ed18['WAITTIME'] = ed18_temp['WAITTIME']

In [5]:
# Imputing the categorical variable 'ARREMS' using mode imputation
for column in ['ARREMS']:
    ed18[column].fillna(ed18[column].mode()[0], inplace = True)
ed18['ARREMS'] = ed18['ARREMS'].replace([3], 2)
    
# Imputing the categorical variable 'AMBTRANSFER' using mode imputation
for column in ['AMBTRANSFER']:
    ed18[column].fillna(ed18[column].mode()[0], inplace = True)

In [6]:
from sklearn.experimental import enable_iterative_imputer # Required to import Iterative Imputer package
from sklearn.impute import IterativeImputer # Importing the iterative imputer package used by MICE

# Defining the imputer
mice_imputer = IterativeImputer()

# Fitting and transforming the imputed values and importing it into a temporary dataframe
ed18filled = mice_imputer.fit_transform(ed18)
ed18filled_temp = pd.DataFrame(ed18filled)

# Matching the temporary dataframe columns with the original dataframe
ed18filled_temp.columns = ed18.columns

# Replacing missing values in the original dataframe with imputed values from the temporary dataframe
ed18['IMMEDR'] = ed18filled_temp['IMMEDR']
ed18['PAINSCALE'] = ed18filled_temp['PAINSCALE']

In [7]:
# Rounding up the imputed results to the nearest integer

ed18['WAITTIME'] = ed18['WAITTIME'].round()
ed18['IMMEDR'] = ed18['IMMEDR'].round()
ed18['PAINSCALE'] = ed18['PAINSCALE'].round()

# Printing temporary dataframe to view the changes
ed18[:20]

,WAITTIME,AGER,SEX,ARREMS,AMBTRANSFER,IMMEDR,PAINSCALE,HOSPCODE,PATCODE
0,21.0,1.0,1.0,0.0,3.0,4.0,0.0,1.0,1.0
1,12.0,1.0,0.0,0.0,3.0,4.0,6.0,1.0,2.0
2,21.0,1.0,1.0,0.0,3.0,4.0,4.0,1.0,3.0
3,59.0,2.0,0.0,0.0,3.0,4.0,10.0,1.0,4.0
4,25.0,3.0,0.0,0.0,3.0,4.0,10.0,1.0,5.0
5,8.0,1.0,1.0,0.0,3.0,4.0,0.0,1.0,6.0
6,16.0,1.0,0.0,0.0,3.0,3.0,4.0,1.0,7.0
7,12.0,4.0,0.0,0.0,3.0,4.0,5.0,1.0,8.0
8,7.0,1.0,0.0,0.0,3.0,4.0,0.0,1.0,9.0
9,19.0,4.0,0.0,0.0,3.0,3.0,0.0,1.0,10.0


In [8]:
# Printing descriptive analysis of the cleaned dataset
print("\nDescriptive Analysis of Waiting Time for Test Dataset\n")
descanalysisclean = ed18['WAITTIME'].describe()
print(descanalysisclean)

# Printing the number of missing values to double check
print("\nNumber of Missing Values Found for Each Variable\n")
newmiss = ed18.isnull().sum()
print(newmiss)


Descriptive Analysis of Waiting Time for Test Dataset

count    20291.000000
mean        36.311172
std         66.376932
min         -7.000000
25%          6.000000
50%         17.000000
75%         41.000000
max       1282.000000
Name: WAITTIME, dtype: float64

Number of Missing Values Found for Each Variable

WAITTIME       0
AGER           0
SEX            0
ARREMS         0
AMBTRANSFER    0
IMMEDR         0
PAINSCALE      0
HOSPCODE       0
PATCODE        0
dtype: int64


In [ ]:
# Transforming the values in the variable wait time to a categorical value
ed18.loc[(ed18['WAITTIME'] >= 0) & (ed18['WAITTIME'] <= 10), 'WAITTIME'] = 1
ed18.loc[(ed18['WAITTIME'] > 10) & (ed18['WAITTIME'] <= 60), 'WAITTIME'] = 2
ed18.loc[(ed18['WAITTIME'] > 60), 'WAITTIME'] = 3
ed18.loc[(ed18['WAITTIME'] < 0), 'WAITTIME'] = 0

In [ ]:
# Writing the clean dataset into a .csv file
ed18.to_csv('C:/Users/Kenneth/Desktop/18040394_KennethLauKhengSoong_Aug21/System Files/Datasheets/ED18.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

# Finding the values of waiting time
wtt = ed18['WAITTIME'].value_counts()
print(wtt)

# Plotting a pie chart for waiting time classification
waittimelabel = ['Not Seen', 'Short', 'Medium', 'Long']
waittimedata = [593, 6935, 9500, 3263]
explode = (0.0, 0.0, 0.075, 0.0)

fig = plt.figure(figsize = (10, 7))
plt.pie(waittimedata,
        labels = waittimelabel,
        explode = explode,
        shadow = True)

plt.title("Classification of Patient's Waiting Times")
plt.show()

In [ ]:
# Plotting a histogram for patient's immediacy to be assigned to a triage
immedrax = ed18.hist(column='IMMEDR',
                     bins = 7,
                     grid = False,
                     figsize = (10, 7))

immedrax = immedrax[0]
for x in immedrax:
    x.set_title("Patient's Immediacy to be Assigned a Triage",
                weight = 'bold',
                size = 12)
    x.set_xlabel("Immediacy Level",
                 weight = 'bold',
                 size = 12)
    x.set_ylabel("Count",
                 weight = 'bold',
                 size = 12)

In [ ]:
# Plotting a histogram for patient's described pain scale
psax = ed18.hist(column='PAINSCALE',
                     bins = 10,
                     grid = False,
                     figsize = (10, 7))

psax = psax[0]
for x in psax:
    x.set_title("Patient's Pain Scale",
                weight = 'bold',
                size = 12)
    x.set_xlabel("Pain Scale Level",
                 weight = 'bold',
                 size = 12)
    x.set_ylabel("Count",
                 weight = 'bold',
                 size = 12)